In [ ]:
import random
from collections import Counter
import json
import os

class GenshinGachaSimulator:
    def __init__(self):
        # 基础概率设置
        self.base_rates = {
            "5_star": 0.006,  # 0.6%
            "4_star": 0.051,  # 5.1%
            "3_star": 0.943   # 94.3%
        }
        
        # 软保底和硬保底设置
        self.hard_pity_5_star = 90    # 90抽必出5星
        self.soft_pity_start_5_star = 74  # 74抽开始概率提升
        self.hard_pity_4_star = 10    # 10抽必出4星
        
        # 卡池设置
        self.character_pool = {
            "5_star_up": ["纳西妲", "妮露"],  # UP五星角色
            "5_star_standard": ["迪卢克", "刻晴", "莫娜", "七七", "琴"],  # 常驻五星角色
            "4_star_up": ["久岐忍", "多莉", "莱依拉"],  # UP四星角色
            "4_star_standard": ["芭芭拉", "菲谢尔", "班尼特", "行秋", "香菱"],  # 其他四星角色
            "3_star": ["魔导绪论", "鸦羽弓", "沐浴龙血的剑", "飞天御剑", "黑缨枪"]  # 三星武器
        }
        
        # 计数器和状态
        self.pity_counter_5_star = 0
        self.pity_counter_4_star = 0
        self.guaranteed_up_5_star = False  # 是否大保底
        self.wish_history = []
        
        # 统计数据
        self.total_wishes = 0
        self.stats = {
            "5_star_count": 0,
            "4_star_count": 0,
            "3_star_count": 0,
            "primogems_spent": 0
        }
    
    def wish(self, num_wishes=1):
        """执行抽卡操作"""
        results = []
        for _ in range(num_wishes):
            result = self._perform_single_wish()
            results.append(result)
            self.wish_history.append(result)
            self.total_wishes += 1
            self.stats["primogems_spent"] += 160  # 每抽160原石
            
        return results
    
    def _perform_single_wish(self):
        """执行单次抽卡"""
        self.pity_counter_5_star += 1
        self.pity_counter_4_star += 1
        
        # 检查保底
        if self.pity_counter_5_star >= self.hard_pity_5_star:
            return self._get_guaranteed_5_star()
        
        if self.pity_counter_4_star >= self.hard_pity_4_star:
            return self._get_guaranteed_4_star()
        
        # 计算当前5星概率（软保底）
        current_5_star_rate = self._get_current_5_star_rate()
        
        # 随机决定稀有度
        roll = random.random()
        
        if roll < current_5_star_rate:
            return self._get_5_star()
        elif roll < current_5_star_rate + self.base_rates["4_star"]:
            return self._get_4_star()
        else:
            return self._get_3_star()
    
    def _get_current_5_star_rate(self):
        """计算当前5星概率（软保底机制）"""
        if self.pity_counter_5_star <= self.soft_pity_start_5_star:
            return self.base_rates["5_star"]
        else:
            # 从74抽开始，每抽增加6%概率
            increased_rate = self.base_rates["5_star"] + 0.06 * (self.pity_counter_5_star - self.soft_pity_start_5_star)
            return min(increased_rate, 1.0)  # 确保概率不超过100%
    
    def _get_guaranteed_5_star(self):
        """获取保底5星角色"""
        self.pity_counter_5_star = 0
        self.stats["5_star_count"] += 1
        
        if self.guaranteed_up_5_star or random.random() < 0.5:
            self.guaranteed_up_5_star = False
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_up"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": True
            }
        else:
            self.guaranteed_up_5_star = True
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_standard"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": False
            }
        
        return result
    
    def _get_5_star(self):
        """获取非保底5星角色"""
        self.pity_counter_5_star = 0
        self.stats["5_star_count"] += 1
        
        if self.guaranteed_up_5_star or random.random() < 0.5:
            self.guaranteed_up_5_star = False
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_up"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": self.guaranteed_up_5_star
            }
        else:
            self.guaranteed_up_5_star = True
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_standard"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": self.guaranteed_up_5_star
            }
        
        return result
    
    def _get_guaranteed_4_star(self):
        """获取保底4星角色"""
        self.pity_counter_4_star = 0
        self.stats["4_star_count"] += 1
        
        if random.random() < 0.5:  # 50%概率获得UP四星角色
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_up"]),
                "pity": self.pity_counter_4_star
            }
        else:
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_standard"]),
                "pity": self.pity_counter_4_star
            }
        
        return result
    
    def _get_4_star(self):
        """获取非保底4星角色"""
        self.pity_counter_4_star = 0
        self.stats["4_star_count"] += 1
        
        if random.random() < 0.5:  # 50%概率获得UP四星角色
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_up"]),
                "pity": self.pity_counter_4_star
            }
        else:
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_standard"]),
                "pity": self.pity_counter_4_star
            }
        
        return result
    
    def _get_3_star(self):
        """获取3星物品"""
        self.stats["3_star_count"] += 1
        return {
            "type": "3_star",
            "item": random.choice(self.character_pool["3_star"]),
            "pity": self.pity_counter_4_star
        }
    
    def get_stats(self):
        """获取统计数据"""
        return self.stats
    
    def get_pity_counters(self):
        """获取当前保底计数"""
        return {
            "5_star_pity": self.pity_counter_5_star,
            "4_star_pity": self.pity_counter_4_star,
            "guaranteed_up": self.guaranteed_up_5_star
        }
    
    def get_wish_history(self, limit=10):
        """获取抽卡历史"""
        return self.wish_history[-limit:] if limit else self.wish_history
    
    def save_history(self, filename="wish_history.json"):
        """保存抽卡历史到文件"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump({
                "wish_history": self.wish_history,
                "stats": self.stats,
                "pity_counters": self.get_pity_counters()
            }, f, ensure_ascii=False, indent=4)
    
    def load_history(self, filename="wish_history.json"):
        """从文件加载抽卡历史"""
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.wish_history = data.get("wish_history", [])
                self.stats = data.get("stats", self.stats)
                pity_data = data.get("pity_counters", {})
                self.pity_counter_5_star = pity_data.get("5_star_pity", 0)
                self.pity_counter_4_star = pity_data.get("4_star_pity", 0)
                self.guaranteed_up_5_star = pity_data.get("guaranteed_up", False)

def display_results(results):
    """显示抽卡结果"""
    for result in results:
        if result['type'] == '5_star':
            print(f"★★★★★ {result['item']} (在第{result['pity']}抽获得)")
        elif result['type'] == '4_star':
            print(f"★★★★ {result['item']} (在第{result['pity']}抽获得)")
        else:
            print(f"★★★ {result['item']}")

def main():
    simulator = GenshinGachaSimulator()
    
    # 尝试加载历史记录
    if os.path.exists("wish_history.json"):
        load = input("检测到历史记录，是否加载？(y/n): ")
        if load.lower() == 'y':
            simulator.load_history()
            print("历史记录已加载！")
    
    print("原神抽卡模拟器")
    print("=" * 50)
    
    while True:
        print(f"\n当前5星保底: {simulator.get_pity_counters()['5_star_pity']}/90")
        print(f"当前4星保底: {simulator.get_pity_counters()['4_star_pity']}/10")
        if simulator.get_pity_counters()['guaranteed_up']:
            print("下次5星必定为UP角色!")
        
        print("\n1. 单抽")
        print("2. 十连抽")
        print("3. 显示统计数据")
        print("4. 显示最近抽卡记录")
        print("5. 保存记录")
        print("6. 退出")
        
        choice = input("请选择操作: ")
        
        if choice == '1':
            results = simulator.wish(1)
            display_results(results)
        
        elif choice == '2':
            results = simulator.wish(10)
            display_results(results)
        
        elif choice == '3':
            stats = simulator.get_stats()
            total_wishes = stats['5_star_count'] + stats['4_star_count'] + stats['3_star_count']
            print(f"\n统计数据:")
            print(f"总抽卡数: {total_wishes}")
            print(f"5星数量: {stats['5_star_count']} ({(stats['5_star_count']/total_wishes*100 if total_wishes > 0 else 0):.2f}%)")
            print(f"4星数量: {stats['4_star_count']} ({(stats['4_star_count']/total_wishes*100 if total_wishes > 0 else 0):.2f}%)")
            print(f"3星数量: {stats['3_star_count']} ({(stats['3_star_count']/total_wishes*100 if total_wishes > 0 else 0):.2f}%)")
            print(f"消耗原石: {stats['primogems_spent']}")
        
        elif choice == '4':
            history = simulator.get_wish_history(20)
            print("\n最近抽卡记录:")
            for i, wish in enumerate(history, 1):
                star = "★★★★★" if wish['type'] == '5_star' else "★★★★" if wish['type'] == '4_star' else "★★★"
                print(f"{i}. {star} {wish['item']}")
        
        elif choice == '5':
            simulator.save_history()
            print("记录已保存到 wish_history.json")
        
        elif choice == '6':
            save = input("退出前是否保存记录？(y/n): ")
            if save.lower() == 'y':
                simulator.save_history()
                print("记录已保存!")
            break
        
        else:
            print("无效选择，请重新输入!")

if __name__ == "__main__":
    main()

原神抽卡模拟器

当前5星保底: 0/90
当前4星保底: 0/10

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


请选择操作:  2


★★★ 魔导绪论
★★★ 魔导绪论
★★★ 鸦羽弓
★★★ 沐浴龙血的剑
★★★ 飞天御剑
★★★ 飞天御剑
★★★ 沐浴龙血的剑
★★★ 沐浴龙血的剑
★★★ 鸦羽弓
★★★★ 芭芭拉 (在第0抽获得)

当前5星保底: 10/90
当前4星保底: 0/10

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


请选择操作:  2


★★★ 沐浴龙血的剑
★★★ 魔导绪论
★★★ 魔导绪论
★★★ 飞天御剑
★★★ 飞天御剑
★★★ 飞天御剑
★★★ 飞天御剑
★★★ 黑缨枪
★★★ 鸦羽弓
★★★★ 菲谢尔 (在第0抽获得)

当前5星保底: 20/90
当前4星保底: 0/10

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


...

★★★ 飞天御剑
★★★ 鸦羽弓
★★★ 鸦羽弓
★★★ 魔导绪论
★★★ 魔导绪论
★★★ 鸦羽弓
★★★ 黑缨枪
★★★ 飞天御剑
★★★★ 久岐忍 (在第0抽获得)
★★★ 黑缨枪

当前5星保底: 70/90
当前4星保底: 1/10

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


请选择操作:  2


★★★ 沐浴龙血的剑
★★★ 飞天御剑
★★★ 鸦羽弓
★★★ 沐浴龙血的剑
★★★ 沐浴龙血的剑
★★★★ 多莉 (在第0抽获得)
★★★ 沐浴龙血的剑
★★★★ 多莉 (在第0抽获得)
★★★★★ 刻晴 (在第0抽获得)
★★★ 魔导绪论

当前5星保底: 1/90
当前4星保底: 2/10
下次5星必定为UP角色!

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


请选择操作:  4



最近抽卡记录:
1. ★★★ 飞天御剑
2. ★★★ 鸦羽弓
3. ★★★ 鸦羽弓
4. ★★★ 魔导绪论
5. ★★★ 魔导绪论
6. ★★★ 鸦羽弓
7. ★★★ 黑缨枪
8. ★★★ 飞天御剑
9. ★★★★ 久岐忍
10. ★★★ 黑缨枪
11. ★★★ 沐浴龙血的剑
12. ★★★ 飞天御剑
13. ★★★ 鸦羽弓
14. ★★★ 沐浴龙血的剑
15. ★★★ 沐浴龙血的剑
16. ★★★★ 多莉
17. ★★★ 沐浴龙血的剑
18. ★★★★ 多莉
19. ★★★★★ 刻晴
20. ★★★ 魔导绪论

当前5星保底: 1/90
当前4星保底: 2/10
下次5星必定为UP角色!

1. 单抽
2. 十连抽
3. 显示统计数据
4. 显示最近抽卡记录
5. 保存记录
6. 退出


In [ ]:
%gui tk  # 在运行前先启用 Tkinter 支持

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import random
import json
import os
from collections import Counter

class GenshinGachaSimulator:
    def __init__(self):
        # 基础概率设置
        self.base_rates = {
            "5_star": 0.006,  # 0.6%
            "4_star": 0.051,  # 5.1%
            "3_star": 0.943   # 94.3%
        }
        
        # 保底设置
        self.hard_pity_5_star = 90    # 90抽必出5星
        self.soft_pity_start_5_star = 74  # 74抽开始概率提升
        self.hard_pity_4_star = 10    # 10抽必出4星
        
        # 卡池设置（使用原神实际角色名称）
        self.character_pool = {
            "5_star_up": ["纳西妲", "妮露"],  # UP五星角色
            "5_star_standard": ["迪卢克", "刻晴", "莫娜", "七七", "琴"],  # 常驻五星角色
            "4_star_up": ["久岐忍", "多莉", "莱依拉"],  # UP四星角色
            "4_star_standard": ["芭芭拉", "菲谢尔", "班尼特", "行秋", "香菱"],  # 其他四星角色
            "3_star": ["魔导绪论", "鸦羽弓", "沐浴龙血的剑", "飞天御剑", "黑缨枪"]  # 三星武器
        }
        
        # 计数器和状态
        self.pity_counter_5_star = 0
        self.pity_counter_4_star = 0
        self.guaranteed_up_5_star = False  # 是否大保底
        self.wish_history = []
        
        # 统计数据
        self.total_wishes = 0
        self.stats = {
            "5_star_count": 0,
            "4_star_count": 0,
            "3_star_count": 0,
            "primogems_spent": 0
        }
    
    def wish(self, num_wishes=1):
        """执行抽卡操作"""
        results = []
        for _ in range(num_wishes):
            result = self._perform_single_wish()
            results.append(result)
            self.wish_history.append(result)
            self.total_wishes += 1
            self.stats["primogems_spent"] += 160  # 每抽160原石
            
        return results
    
    def _perform_single_wish(self):
        """执行单次抽卡"""
        self.pity_counter_5_star += 1
        self.pity_counter_4_star += 1
        
        # 检查保底
        if self.pity_counter_5_star >= self.hard_pity_5_star:
            return self._get_guaranteed_5_star()
        
        if self.pity_counter_4_star >= self.hard_pity_4_star:
            return self._get_guaranteed_4_star()
        
        # 计算当前5星概率（软保底）
        current_5_star_rate = self._get_current_5_star_rate()
        
        # 随机决定稀有度
        roll = random.random()
        
        if roll < current_5_star_rate:
            return self._get_5_star()
        elif roll < current_5_star_rate + self.base_rates["4_star"]:
            return self._get_4_star()
        else:
            return self._get_3_star()
    
    def _get_current_5_star_rate(self):
        """计算当前5星概率（软保底机制）"""
        if self.pity_counter_5_star <= self.soft_pity_start_5_star:
            return self.base_rates["5_star"]
        else:
            # 从74抽开始，每抽增加6%概率
            increased_rate = self.base_rates["5_star"] + 0.06 * (self.pity_counter_5_star - self.soft_pity_start_5_star)
            return min(increased_rate, 1.0)  # 确保概率不超过100%
    
    def _get_guaranteed_5_star(self):
        """获取保底5星角色"""
        self.pity_counter_5_star = 0
        self.stats["5_star_count"] += 1
        
        if self.guaranteed_up_5_star or random.random() < 0.5:
            self.guaranteed_up_5_star = False
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_up"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": True
            }
        else:
            self.guaranteed_up_5_star = True
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_standard"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": False
            }
        
        return result
    
    def _get_5_star(self):
        """获取非保底5星角色"""
        self.pity_counter_5_star = 0
        self.stats["5_star_count"] += 1
        
        if self.guaranteed_up_5_star or random.random() < 0.5:
            self.guaranteed_up_5_star = False
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_up"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": self.guaranteed_up_5_star
            }
        else:
            self.guaranteed_up_5_star = True
            result = {
                "type": "5_star",
                "item": random.choice(self.character_pool["5_star_standard"]),
                "pity": self.pity_counter_5_star,
                "is_guaranteed": self.guaranteed_up_5_star
            }
        
        return result
    
    def _get_guaranteed_4_star(self):
        """获取保底4星角色"""
        self.pity_counter_4_star = 0
        self.stats["4_star_count"] += 1
        
        if random.random() < 0.5:  # 50%概率获得UP四星角色
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_up"]),
                "pity": self.pity_counter_4_star
            }
        else:
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_standard"]),
                "pity": self.pity_counter_4_star
            }
        
        return result
    
    def _get_4_star(self):
        """获取非保底4星角色"""
        self.pity_counter_4_star = 0
        self.stats["4_star_count"] += 1
        
        if random.random() < 0.5:  # 50%概率获得UP四星角色
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_up"]),
                "pity": self.pity_counter_4_star
            }
        else:
            result = {
                "type": "4_star",
                "item": random.choice(self.character_pool["4_star_standard"]),
                "pity": self.pity_counter_4_star
            }
        
        return result
    
    def _get_3_star(self):
        """获取3星物品"""
        self.stats["3_star_count"] += 1
        return {
            "type": "3_star",
            "item": random.choice(self.character_pool["3_star"]),
            "pity": self.pity_counter_4_star
        }
    
    def get_stats(self):
        """获取统计数据"""
        return self.stats
    
    def get_pity_counters(self):
        """获取当前保底计数"""
        return {
            "5_star_pity": self.pity_counter_5_star,
            "4_star_pity": self.pity_counter_4_star,
            "guaranteed_up": self.guaranteed_up_5_star
        }
    
    def get_wish_history(self, limit=10):
        """获取抽卡历史"""
        return self.wish_history[-limit:] if limit else self.wish_history
    
    def save_history(self, filename="wish_history.json"):
        """保存抽卡历史到文件"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump({
                "wish_history": self.wish_history,
                "stats": self.stats,
                "pity_counters": self.get_pity_counters()
            }, f, ensure_ascii=False, indent=4)
    
    def load_history(self, filename="wish_history.json"):
        """从文件加载抽卡历史"""
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.wish_history = data.get("wish_history", [])
                self.stats = data.get("stats", self.stats)
                pity_data = data.get("pity_counters", {})
                self.pity_counter_5_star = pity_data.get("5_star_pity", 0)
                self.pity_counter_4_star = pity_data.get("4_star_pity", 0)
                self.guaranteed_up_5_star = pity_data.get("guaranteed_up", False)
            return True
        return False

class GachaGUI:
    def __init__(self):
        self.simulator = GenshinGachaSimulator()
        self.root = tk.Tk()
        self.root.title("原神抽卡模拟器")
        self.root.geometry("800x600")
        self.root.resizable(True, True)
        
        # 设置样式
        self.setup_styles()
        
        # 创建界面
        self.create_widgets()
        
        # 尝试加载历史记录
        if self.simulator.load_history():
            self.update_display()
    
    def setup_styles(self):
        """设置样式"""
        self.style = ttk.Style()
        self.style.configure("Title.TLabel", font=("Arial", 16, "bold"))
        self.style.configure("Pity.TLabel", font=("Arial", 12))
        self.style.configure("5Star.TLabel", foreground="gold", font=("Arial", 12, "bold"))
        self.style.configure("4Star.TLabel", foreground="purple", font=("Arial", 12))
        self.style.configure("3Star.TLabel", foreground="blue", font=("Arial", 12))
        
        self.style.configure("WishButton.TButton", font=("Arial", 14))
        self.style.configure("ActionButton.TButton", font=("Arial", 10))
    
    def create_widgets(self):
        """创建界面组件"""
        # 主框架
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 标题
        title_label = ttk.Label(main_frame, text="原神抽卡模拟器", style="Title.TLabel")
        title_label.grid(row=0, column=0, columnspan=2, pady=(0, 20))
        
        # 保底信息框架
        pity_frame = ttk.LabelFrame(main_frame, text="保底信息", padding="10")
        pity_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=(0, 10))
        
        # 5星保底
        self.pity_5_label = ttk.Label(pity_frame, text="5星保底: 0/90", style="Pity.TLabel")
        self.pity_5_label.grid(row=0, column=0, sticky=tk.W)
        
        # 4星保底
        self.pity_4_label = ttk.Label(pity_frame, text="4星保底: 0/10", style="Pity.TLabel")
        self.pity_4_label.grid(row=1, column=0, sticky=tk.W)
        
        # 大保底状态
        self.guarantee_label = ttk.Label(pity_frame, text="下次5星: 50%概率UP角色", style="Pity.TLabel")
        self.guarantee_label.grid(row=2, column=0, sticky=tk.W)
        
        # 抽卡按钮框架
        button_frame = ttk.Frame(main_frame)
        button_frame.grid(row=2, column=0, columnspan=2, pady=(0, 10))
        
        # 单抽按钮
        wish_1_button = ttk.Button(button_frame, text="祈愿一次", command=self.wish_1, style="WishButton.TButton")
        wish_1_button.grid(row=0, column=0, padx=5)
        
        # 十连按钮
        wish_10_button = ttk.Button(button_frame, text="祈愿十次", command=self.wish_10, style="WishButton.TButton")
        wish_10_button.grid(row=0, column=1, padx=5)
        
        # 结果显示框架
        result_frame = ttk.LabelFrame(main_frame, text="抽卡结果", padding="10")
        result_frame.grid(row=3, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(0, 10))
        
        # 结果文本区域
        self.result_text = scrolledtext.ScrolledText(result_frame, width=40, height=10, font=("Arial", 10))
        self.result_text.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 历史记录框架
        history_frame = ttk.LabelFrame(main_frame, text="最近抽卡记录", padding="10")
        history_frame.grid(row=3, column=1, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(0, 10))
        
        # 历史记录文本区域
        self.history_text = scrolledtext.ScrolledText(history_frame, width=40, height=10, font=("Arial", 10))
        self.history_text.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 统计信息框架
        stats_frame = ttk.LabelFrame(main_frame, text="统计信息", padding="10")
        stats_frame.grid(row=4, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=(0, 10))
        
        # 统计信息标签
        self.stats_label = ttk.Label(stats_frame, text="总抽卡数: 0 | 5星: 0 (0.00%) | 4星: 0 (0.00%) | 3星: 0 (0.00%)")
        self.stats_label.grid(row=0, column=0, sticky=tk.W)
        
        # 原石消耗标签
        self.primo_label = ttk.Label(stats_frame, text="消耗原石: 0")
        self.primo_label.grid(row=1, column=0, sticky=tk.W)
        
        # 操作按钮框架
        action_frame = ttk.Frame(main_frame)
        action_frame.grid(row=5, column=0, columnspan=2, pady=(10, 0))
        
        # 保存按钮
        save_button = ttk.Button(action_frame, text="保存记录", command=self.save_data, style="ActionButton.TButton")
        save_button.grid(row=0, column=0, padx=5)
        
        # 重置按钮
        reset_button = ttk.Button(action_frame, text="重置模拟器", command=self.reset_simulator, style="ActionButton.TButton")
        reset_button.grid(row=0, column=1, padx=5)
        
        # 退出按钮
        exit_button = ttk.Button(action_frame, text="退出", command=self.exit_app, style="ActionButton.TButton")
        exit_button.grid(row=0, column=2, padx=5)
        
        # 配置网格权重
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(3, weight=1)
        result_frame.columnconfigure(0, weight=1)
        result_frame.rowconfigure(0, weight=1)
        history_frame.columnconfigure(0, weight=1)
        history_frame.rowconfigure(0, weight=1)
    
    def wish_1(self):
        """执行单次抽卡"""
        results = self.simulator.wish(1)
        self.show_results(results)
        self.update_display()
    
    def wish_10(self):
        """执行十连抽卡"""
        results = self.simulator.wish(10)
        self.show_results(results)
        self.update_display()
    
    def show_results(self, results):
        """显示抽卡结果"""
        self.result_text.delete(1.0, tk.END)
        
        for result in results:
            if result['type'] == '5_star':
                star_text = "★★★★★"
                color_tag = "five_star"
            elif result['type'] == '4_star':
                star_text = "★★★★"
                color_tag = "four_star"
            else:
                star_text = "★★★"
                color_tag = "three_star"
            
            result_line = f"{star_text} {result['item']} (第{result['pity']}抽)\n"
            self.result_text.insert(tk.END, result_line)
        
        # 滚动到最底部
        self.result_text.see(tk.END)
    
    def update_display(self):
        """更新界面显示"""
        # 更新保底信息
        pity = self.simulator.get_pity_counters()
        self.pity_5_label.config(text=f"5星保底: {pity['5_star_pity']}/90")
        self.pity_4_label.config(text=f"4星保底: {pity['4_star_pity']}/10")
        
        if pity['guaranteed_up']:
            self.guarantee_label.config(text="下次5星: 100%概率UP角色（大保底）")
        else:
            self.guarantee_label.config(text="下次5星: 50%概率UP角色")
        
        # 更新统计信息
        stats = self.simulator.get_stats()
        total = stats['5_star_count'] + stats['4_star_count'] + stats['3_star_count']
        
        if total > 0:
            rate_5 = stats['5_star_count'] / total * 100
            rate_4 = stats['4_star_count'] / total * 100
            rate_3 = stats['3_star_count'] / total * 100
        else:
            rate_5 = rate_4 = rate_3 = 0
        
        self.stats_label.config(
            text=f"总抽卡数: {total} | "
                 f"5星: {stats['5_star_count']} ({rate_5:.2f}%) | "
                 f"4星: {stats['4_star_count']} ({rate_4:.2f}%) | "
                 f"3星: {stats['3_star_count']} ({rate_3:.2f}%)"
        )
        
        self.primo_label.config(text=f"消耗原石: {stats['primogems_spent']}")
        
        # 更新历史记录
        self.update_history()
    
    def update_history(self):
        """更新历史记录显示"""
        self.history_text.delete(1.0, tk.END)
        history = self.simulator.get_wish_history(20)
        
        for i, wish in enumerate(reversed(history), 1):
            if wish['type'] == '5_star':
                star_text = "★★★★★"
            elif wish['type'] == '4_star':
                star_text = "★★★★"
            else:
                star_text = "★★★"
            
            history_line = f"{len(history) - i + 1}. {star_text} {wish['item']}\n"
            self.history_text.insert(tk.END, history_line)
        
        # 滚动到最底部
        self.history_text.see(tk.END)
    
    def save_data(self):
        """保存数据"""
        self.simulator.save_history()
        messagebox.showinfo("保存成功", "抽卡记录已保存到文件！")
    
    def reset_simulator(self):
        """重置模拟器"""
        if messagebox.askyesno("确认重置", "确定要重置模拟器吗？所有记录将被清除！"):
            self.simulator = GenshinGachaSimulator()
            self.update_display()
            messagebox.showinfo("重置成功", "模拟器已重置！")
    
    def exit_app(self):
        """退出应用"""
        if messagebox.askyesno("确认退出", "确定要退出吗？请确保已保存需要的数据！"):
            self.root.destroy()
    
    def run(self):
        """运行应用"""
        self.root.mainloop()

if __name__ == "__main__":
    app = GachaGUI()
    app.run()